In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import multiprocessing
import time
import argparse

%matplotlib qt5
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

In [8]:
path = r'E:\data\CM-4M-o18-2020.10.17-1-60'
path = path.replace('\\','/')
os.chdir(path)
file_list = os.listdir(path)
os.getcwd()

'E:\\data\\CM-4M-o18-2020.10.17-1-60'

In [72]:
data_path = 'CM-4M-o18-2020.10.17-1-60.txt'
amplitude = []
energy = []
duration = []
rise_time = []
rms = []
with open(data_path, "r") as f:
    f.readline()
    for _ in range(1088):
        data = f.readline().split(',')
        amplitude.append(float(data[6].strip()))
        rise_time.append(float(data[7].strip()))
        duration.append(float(data[8].strip()))
        energy.append(float(data[9].strip()))
print('amplitude: ', min(amplitude), max(amplitude))
print('duration: ', min(duration), max(duration))
print('rise_time: ', min(rise_time), max(rise_time))
print('energy: ', min(energy), max(energy))

amplitude:  35.2 40.1
duration:  0.0008056 0.0010238
rise_time:  0.0 0.0010234
energy:  0.83619549 1.73383307


In [69]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax1.scatter(amplitude, energy, marker='o',c='',edgecolors='g', label = 'Amplitude-Energy')
# ax2 = ax1.twinx()
# ax2.scatter(amplitude, rms, marker='o',c='',edgecolors='b', label = 'Amplitude-RMS')
ax1.set_xlabel('Amplitude /μV')
ax1.set_ylabel('Energy /aJ')
ax1.set_title("Amplitude - Energy")
# ax1.set_ylim(0, pow(10, 7))
# ax1.legend(loc=0)
# ax2.set_ylabel('RMS /μV')
# ax2.set_ylim(0, 50)
# ax2.legend(loc=0)
ax1.set_xlim(35, 41)
ax1.set_xticks(np.linspace(35, 41, 7))
ax1.set_ylim(0.75, 2.00)
ax1.set_yticks(np.linspace(0.75, 2.00, 6))
ax1.legend(loc='upper left')
ax1.grid(True,linestyle = "--",color = 'gray' ,linewidth = '0.5',axis='both')

ax3 = fig.add_subplot(212)
ax3.scatter(duration, amplitude, marker='o',c='',edgecolors='purple', label = 'Duration-Amplitude')
ax3.set_xlabel('Duration /s')
ax3.set_ylabel('Amplitude /μV')
ax3.set_title("Duration - Amplitude")
ax3.set_xlim(0.00080, 0.00105)
ax3.set_xticks(np.linspace(0.00080, 0.00105, 6))
ax3.set_ylim(35, 41)
ax3.set_yticks(np.linspace(35, 41, 7))
ax3.legend(loc='upper left')
ax3.grid(True,linestyle = "--",color = 'gray' ,linewidth = '0.5',axis='both')

plt.tight_layout()

<ipython-input-69-6f4ad707beed>:7: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  ax1.scatter(amplitude, energy, marker='o',c='',edgecolors='g', label = 'Amplitude-Energy')
<ipython-input-69-6f4ad707beed>:26: MatplotlibDeprecationWarning: Using a string of single character colors as a color sequence is deprecated since 3.2 and will be removed two minor releases later. Use an explicit list instead.
  ax3.scatter(duration, amplitude, marker='o',c='',edgecolors='purple', label = 'Duration-Amplitude')


In [50]:
file = r'E:\data\CM-4M-o18-2020.10.17-1-60\CM-4M-o18-2020.10.17-1-60_1_29084_515519344.txt'
# C:\\Users\\Yuan\\Desktop\\waveform_558.txt
# E:\data\CM-4M-o18-2020.10.17-1-60\CM-4M-o18-2020.10.17-1-60_1_48123_876621597.txt
# time = []
# amp = []
with open(file, "r") as f:
    for _ in range(12):
        f.readline()
    amp = np.array([float(i.strip("\n")) for i in f.readlines()])
#     f.readline()
#     for i in f.readlines():
#         time.append(float(i.split(',')[0]))
#         amp.append(float(i.split(',')[1]))
# time = np.array(time)
# amp = np.array(amp)
# thr_μV = 11.3694478249001
time = np.linspace(876.6215975, 876.6215975 + 0.0000002 * 5119, 5120)
thr_1V = pow(10, 25 / 20) / pow(10, 3)
thr_2V = pow(10, 25 / 20) / (5 * pow(10, 3))

def cal_features(dataset, time_label, thr_V):
    time = time_label[-1] - time_label[0]
    valid_wave_idx = np.where(abs(dataset) >= thr_V)[0]
    if valid_wave_idx.shape[0] <= 1:
        return "RuntimeWarning"
    start = time_label[valid_wave_idx[0]]
    end = time_label[valid_wave_idx[-1]]
    duration = end - start
    max_idx = np.argmax(abs(dataset))
    amplitude = abs(dataset[max_idx])
    rise_time = time_label[max_idx] - start
    valid_data = dataset[valid_wave_idx[0]:(valid_wave_idx[-1] + 1)]
    energy = np.sum(np.multiply(pow(valid_data, 2), 0.0000002))
    RMS = math.sqrt(energy / duration)
    counts = cal_counts(valid_data, thr_V)
    return valid_data, duration, amplitude, rise_time, energy, RMS, counts

def cal_counts(valid_data, thr_V):
    counts = 0
    N = len(valid_data)
    for idx in range(1, N):
        if valid_data[idx - 1] <= thr_V <= valid_data[idx]:
            counts += 1
    return counts

valid_data, duration, amplitude, rise_time, energy, RMS, counts = cal_features(amp, time, thr_2V)
print('duration(s), amplitude(V), rise_time(s), energy(aJ), RMS(V), counts')
print(duration, amplitude, rise_time, energy * pow(10, 14), RMS, counts)

duration(s), amplitude(V), rise_time(s), energy(aJ), RMS(V), counts
0.001023799999984476 0.01358032 0.0008308000000170068 1352519.791532888 0.0036346638524236627 245


### Softmax

In [ ]:
x = np.linspace(-10,10,1000)
y = np.exp(x) / np.sum(np.exp(x))
y2 = np.exp(x / 5) / np.sum(np.exp(x / 5))
y3 = np.exp(x / 10) / np.sum(np.exp(x / 10))
plt.figure()
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')   
ax.yaxis.set_ticks_position('left')          # 指定下边的边作为 x 轴 指定左边的边为 y 轴
ax.spines['bottom'].set_position(('data', 0))   #指定 data 设置的bottom(也就是指定的x轴)绑定到y轴的0这个点上
ax.spines['left'].set_position(('data', 0))
plt.ylim(0, 0.02)
plt.xlim(-10, 10)
plt.plot(x, y, color='b', label='Origin')
plt.plot(x, y2, color='g', label='T=5')
plt.plot(x, y3, color='purple', label='T=10')
plt.legend(loc=0)